### Extract and clean time series data

In [1]:
from utils import fetch_log_returns
training_dataset = fetch_log_returns(start = '1995-01-01', end ='1995-12-31')

[*********************100%***********************]  5 of 5 completed


In [2]:
training_dataset.head()

,sp500,dax,ftse,nikkei,ibex
Date,,,,,
1995-01-05 00:00:00+00:00,-0.000803,-0.010397,-0.006345,-0.003457,-0.019565
1995-01-06 00:00:00+00:00,0.000738,0.003280,0.010726,-0.004939,0.000000
1995-01-09 00:00:00+00:00,0.000326,-0.002330,-0.003006,-0.003826,-0.014346
1995-01-10 00:00:00+00:00,0.001843,0.004213,0.001504,0.002903,-0.005274
1995-01-11 00:00:00+00:00,-0.000043,-0.000155,-0.003601,0.002408,-0.010125


### Define the problem instance and model parameters

In [3]:
from model_params import LAMBDA_1, LAMBDA_2, LAMBDA_3, NLAYERS, NSHOTS, NUM_ASSETS, SIGMA_TARGET, TWO_QUBIT_GATES, K, N, RISK_FREE_RATE


### Build the circuit Ansatz (without angle embedding)

In [4]:
from ansatz import build_hardware_efficient_ansatz
from ansatz import compute_number_of_params
ansatz = build_hardware_efficient_ansatz(num_qubits=N, num_layers=NLAYERS, params=[0]*compute_number_of_params(N,NLAYERS))
print('Hardware efficient circuit ansatz')
print()
print(ansatz.draw())


Hardware efficient circuit ansatz

q0: ─U2─U2─o─────────────────U1─U2─o─────────────────U1─M─
q1: ─U2─U2─X─o───────────────U1─U2─X─o───────────────U1─M─
q2: ─U2─U2───X─o─────────────U1─U2───X─o─────────────U1─M─
q3: ─U2─U2─────X─o───────────U1─U2─────X─o───────────U1─M─
q4: ─U2─U2───────X─o─────────U1─U2───────X─o─────────U1─M─
q5: ─U2─U2─────────X─o───────U1─U2─────────X─o───────U1─M─
q6: ─U2─U2───────────X─o─────U1─U2───────────X─o─────U1─M─
q7: ─U2─U2─────────────X─o───U1─U2─────────────X─o───U1─M─
q8: ─U2─U2───────────────X─o─U1─U2───────────────X─o─U1─M─
q9: ─U2─U2─────────────────X─U1─U2─────────────────X─U1─M─


### Build the Portfolio Optimization Hamiltonian and run the optimization process


In [17]:
import numpy as np
from qibo.optimizers import optimize
from cost_function import compute_total_energy

# Optimize starting from a random guess for the variational parameters
initial_params = np.random.uniform(0, 2*np.pi, compute_number_of_params(N,NLAYERS))

# perform optimization
best, optimal_params, extra = optimize(compute_total_energy(parameters=initial_params,dataset=training_dataset), initial_params)

# set final solution to circuit instance
print('Ground state energy:', best)
print()
print('Optimal parameters', optimal_params)
print()
print('Optimization process info', extra)

TypeError: 'numpy.float64' object is not callable

### Results parsing


In [6]:
from results_parsing import get_optimal_binary_portfolios_prob_and_energy

optimized_ansatz = ansatz.set_parameters(optimal_params)
optimal_binary_portfolios = get_optimal_binary_portfolios_prob_and_energy(ansatz, training_dataset)


In [7]:
optimal_binary_portfolios

{'0000001110': {'stat_freq': 0.01, 'energy': 563.8734615202803},
 '0000101100': {'stat_freq': 0.01, 'energy': 563.8777181830939},
 '0000111000': {'stat_freq': 0.01, 'energy': 563.9228291499591},
 '0000111101': {'stat_freq': 0.01, 'energy': 351.46410189183257},
 '0001000101': {'stat_freq': 0.01, 'energy': 451.41346028289877},
 '0001001100': {'stat_freq': 0.02, 'energy': 506.0631886233391},
 '0001001101': {'stat_freq': 0.02, 'energy': 399.8521259925734},
 '0001001111': {'stat_freq': 0.02, 'energy': 351.4340929499151},
 '0001011110': {'stat_freq': 0.01, 'energy': 351.44260789518415},
 '0001100100': {'stat_freq': 0.01, 'energy': 506.08574883705677},
 '0001100101': {'stat_freq': 0.04, 'energy': 399.87468620638737},
 '0001100110': {'stat_freq': 0.02, 'energy': 451.4177180974767},
 '0001100111': {'stat_freq': 0.02, 'energy': 351.4566531637411},
 '0001101001': {'stat_freq': 0.01, 'energy': 451.43601654213955},
 '0001101101': {'stat_freq': 0.02, 'energy': 351.43835191613465},
 '0001101111': {'s

In [9]:
from results_parsing import get_binary_portfolio, get_minimum_energy

min_energy = get_minimum_energy(optimal_binary_portfolios)
min_energy_portfolio = ''
for portfolio, data in optimal_binary_portfolios.items(): 
    if data['energy'] == min_energy: 
        min_energy_portfolio = portfolio
print(min_energy_portfolio)
# binary_asset_weights = get_binary_portfolio()(training_dataset.columns, optimal_bi)
# binary_asset_weights

1111110101


In [ ]:



get_asset_weight_decimal(binary_asset_weights['nikkei'])

0.0

In [ ]:

portfolio

{'sp500': 0.375, 'dax': 0.375, 'ftse': 0.375, 'nikkei': 0.0, 'ibex': 0.125}

### Calculating portfolio metrics

In [ ]:

get_portfolio_metrics(portfolio, training_dataset)

{'Returns': 0.18201864332913661,
 'Volatility': 0.0832295963967742,
 'Sharpe Ratio': 1.8264974229170783,
 'Normalized Weights': array([0.3, 0.3, 0.3, 0. , 0.1])}

In [ ]:
import matplotlib.pyplot as plt
sizes = optimal_portfolio_brute_force.values()
fig, ax = plt.subplots()
ax.pie(sizes, labels=list(optimal_portfolio_brute_force.keys()), autopct='%1.1f%%')
plt.show()

Parte fácil NO guiada: 
- Resolver por fuerza bruta para ver lo mala que es la optimización pero lo precisa que es la formulación

Para hacer por libre si sabes mucho: 

- Trainable -> unsupervised quantum machine learning
